# About
- Web site : https://sirius.inserjeunes.beta.gouv.fr/verbatims/moderation
- Github : https://github.com/mission-apprentissage/sirius-ml
- Author : Do HUYNH ([@huynhdoo](https://github.com/huynhdoo))
- Date : 2025.03
- Licence : MIT

# Dependencies

In [1]:
from google.colab import userdata
import os
os.environ['SIRIUS_MISTRAL_API_KEY'] = userdata.get('SIRIUS_MISTRAL_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('SIRIUS_HF')

In [2]:
%%writefile requirements.txt
langchain==0.2.14
langchain-mistralai==0.1.12
datasets==3.0.0
huggingface_hub>=0.22.0

Overwriting requirements.txt


In [3]:
%%capture
!pip install -r requirements.txt

# Dataset

In [4]:
# Load dataset
from datasets import load_dataset
dataset = load_dataset("apprentissage-sirius/verbatims", split='all')

In [5]:
import pandas as pd
df = dataset.to_pandas()
pd.DataFrame(df.sample(5)['text'].apply(lambda x:x.split(':')[1].strip()))

,text
4013,sa a été très compliquer de trouver mais il fa...
4680,On apprend à remettre un élément de carrosseri...
8959,En vrai selon les entreprises ça peut être com...
9972,Bla bla
7006,Les journée sont comme au lycée


In [10]:
# Verbatims GEM (pépites)
filter = df['status'].isin(['GEM'])
gems = df[filter]
print(f"{len(gems)} gems: \n")
samples = pd.DataFrame(gems.sample(5)['text'].apply(lambda x:x.split(':')[1].strip()))
print("\n\n".join(samples['text']))

659 gems: 

Je suis coiffeuse donc je coupe les cheveux je fais des couleurs et d’autres technique je participe à l’embellissement d’autrui , j’encaisse je conseille. Je sait tenir mon lieu de réveil propre

Être actif ! Postuler à beaucoup d’endroit qui te plaît, relancer souvent pour montrer que tu es intéressé et s’intéresser en prenant des multiples renseignements.

Tu apprend des choses de ton métier qui t’intéresse et tu approfondit tes connaissances

Beh alors là actuellement on vois le fonctionnement des machines agricoles tels que la fanneuse la faucheuse la presse à balles carré etc et donc en cours on comprends le fonctionnement de chaque engin ainsi que la composition des outils a quoi il serve mais pour comprends il faut faire des stages dans des lycées

Le fait de gagner son propre argent en motivant et cela donne envie de travailler, la seul chose qui manque sont vraiment les vacances scolaires


# LLM Tools

In [7]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import AIMessage, HumanMessage
import os
import json

In [8]:
# RAG tools
from langchain.text_splitter import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI
from google.colab import userdata

def prompt_mistralai(prompt, documents):
    # Creating chain
    question_prompt = PromptTemplate.from_template(
        """
            Produce a """ + prompt + """ only from the following text:
            ----------------
            {text}
            ----------------
            ANSWER: <ANSWER IN FRENCH WITHOUT INTRODUCTION>
        """
    )
    refine_prompt = PromptTemplate.from_template(
        """
        Your job is to produce a """ + prompt + """.
        You have provided an existing answer up to a certain point:
        -----------------
        {existing_answer}
        -----------------
        You have the opportunity to refine the existing answer only with some more context below.
        -----------------
        {text}
        -----------------
        Given the new context, refine your answer
        If the context isn't useful, return your original answer.
        ANSWER: <ANSWER IN FRENCH WITHOUT INTRODUCTION>
        """
    )
    chain = load_summarize_chain(
        llm=ChatMistralAI(mistral_api_key=userdata.get("MISTRAL_API_KEY")),
        chain_type="refine",
        question_prompt=question_prompt,
        refine_prompt=refine_prompt,
        return_intermediate_steps=True,
        input_key="input_documents",
        output_key="output_text",
    )
    return chain.invoke({"input_documents": documents}, return_only_outputs=True)['intermediate_steps'][0]

In [9]:
# Exposition des verbatims
def expose_function(testimony, categories=''):
    if categories == '':
        categories = {
            'Intégration et ambiance': 'oui|non',
            'Apprentissage du métier': 'oui|non',
            'Horaires': 'oui|non',
            'Rythme entreprise - établissement scolaire': 'oui|non',
            'Avoir moins de vacances': 'oui|non',
            'Journée-type en entreprise' : 'oui|non',
            'Ambiance établissement': 'oui|non',
            'Difficulté des cours': 'oui|non',
            'Enseignement proposé par établissement': 'oui|non',
            'Equipements mis à disposition': 'oui|non',
            'Accessibilité pour les personnes en situation de handicap': 'oui|non',
            'Charge de travail': 'oui|non',
            'Journée-type en établissement' : 'oui|non',
        }

    AIPrompt = f"""
        Indiquer si un témoignage aborde un ou plusieurs sujets en fonction des catégories suivantes:
        ```
        - {'|'.join(list(categories.keys()))}
        ```

        Votre réponse doit obligatoirement être au format JSON suivant:
        {str(categories)}

        Si le témoignage ne fournit pas assez d'informations ou avec un contenu non-significatif,
        renvoyer une réponse négative pour toutes les catégories.

        Votre réponse en JSON sans commentaire ou justification:
    """

    UserPrompt = f"""
        Voici un témoignage à catégoriser:
        ```
        {testimony}
        ```
    """
    chat = ChatMistralAI(mistral_api_key=os.environ['SIRIUS_MISTRAL_API_KEY']).bind(response_format={"type": "json_object"})
    response = chat.invoke(
        [
            AIMessage(content=AIPrompt),
            HumanMessage(content=UserPrompt)
        ])
    try:
        return json.loads(response.content)
    except:
        return {key:"non" for key in categories.keys()}

# Pépites

In [ ]:
documents = [Document(text) for text in gems['text']]
rules = prompt_mistralai("Lister les 10 principaux points positifs et négatifs des avis", documents)
print(rules)

In [ ]:
response = prompt_mistralai("Lister les 10 principales thématiques abordées en français", documents)
print(response)

In [16]:
features = f"""
    1. Avantages de l'apprentissage: plus de liberté, épanouissement, formation pratique, préparation à la vie professionnelle, salaire, etc.
    2. Comparaison entre études classiques et apprentissage: maturité, autonomie, compréhension, etc.
    3. Organisation de l'apprentissage: temps partagé entre CFA et entreprise, moins de vacances.
    4. Apprentissage et erreurs: normal de faire des erreurs, c'est comme ça qu'on apprend.
    5. Expérience professionnelle: moments en entreprise pour un réel savoir et une réelle expérience.
    6. Gestion de l'argent: gérer une entrée de salaire, économiser.
    7. Autonomie et indépendance: gagner en autonomie, gagner son propre salaire.
    8. Rémunération: toucher un salaire pendant l'apprentissage.
    9. Reconversion professionnelle: se former à autre chose sans perte financière conséquente.
    10. Recherche d'entreprise: préparation, candidater directement, dialogue oral, être soi-même, etc.
"""

features2 = """
    Points positifs :
    1. L'apprentissage permet de se concentrer sur le métier qui correspond à l'apprenti, avec des formations adaptées à ses besoins et sans cours superflus.
    2. Cela permet une entrée progressive dans le monde du travail, en alternant entre le CFA et l'entreprise, ce qui facilite la préparation aux changements.
    3. L'apprenti acquiert une expérience pratique et des compétences dans le métier choisi.
    4. L'apprenti perçoit un salaire, ce qui lui permet de profiter avec ses amis et de gérer son argent.
    5. L'apprentissage offre une plus grande autonomie et maturité.
    6. L'alternance entre l'école et l'entreprise permet de bénéficier d'un accompagnement et d'une expérience professionnelle.
    7. L'apprentissage peut être une alternative intéressante pour ceux qui ont des difficultés avec les études théoriques.
    8. Les apprentis développent une meilleure compréhension de la vie professionnelle et des responsabilités qui y sont liées.
    9. L'apprentissage offre la possibilité de se former à un nouveau métier sans perte financière conséquente.
    10. Les apprentis acquièrent une expérience concrète et assimilent plus rapidement les concepts théoriques en les mettant en pratique.

    Points négatifs :
    1. L'apprentissage implique moins de vacances, généralement seulement 5 semaines par an.
    2. Les apprentis peuvent être stressés par les erreurs commises pendant leur apprentissage.
    3. Certaines entreprises peuvent avoir des attentes irréalistes vis-à-vis des apprentis.
    4. Les apprentis peuvent manquer de connaissances sur les lois et le code du travail, ce qui peut entraîner des problèmes liés aux heures supplémentaires, aux congés et aux horaires de travail.
    5. La recherche d'une entreprise d'accueil peut être difficile et décourageante, surtout pour les femmes dans des métiers traditionnellement masculins.
    6. Les apprentis peuvent souffrir d'un manque de matières générales telles que l'anglais, les mathématiques ou l'histoire-géographie.
    7. Les horaires de travail peuvent être longs et exigeants, avec des journées pouvant durer de 8h à 17h, voire plus, en fonction du métier et de l'entreprise.
    8. Les cours théoriques peuvent être ennuyeux et difficiles à suivre pour certains apprentis, en particulier lorsqu'ils durent plusieurs heures d'affilée.
    9. Les apprentis peuvent se retrouver avec des tâches difficiles dès le début de leur formation, ce qui peut être intimidant.
    10. L'intégration dans une nouvelle équipe peut être difficile, surtout lorsqu'on débute dans le monde professionnel.
"""

from langchain_core.messages import AIMessage, HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI
from google.colab import userdata
import json

FORMAT ="""
        {
            "avis": "oui|non"
            "score": "score en fonction du nombre de critères d'intérêts évoqués, format float précision 2 chiffres"
            "justification": "justification de l'avis"
        }
        """

def gem_classifier(rules, testimony):
    AIPrompt = f"""
        Vous êtes un modérateur qui doit publier ou non un témoignage d'expérience positive ou négative en fonction de son intérêt.
        Vos critères d'intérêts sont:
        ```
        {rules}
        ```
        Si le témoignage ne fournit pas assez d'informations, ne pas le publier.
        Votre réponse doit être conforme au format JSON suivant:
        {FORMAT}

        Votre réponse en JSON:
    """

    UserPrompt = f"""
        Voici un témoignage à modérer:
        ```
        {testimony}
        ```
    """
    chat = ChatMistralAI(mistral_api_key=userdata.get("MISTRAL_API_KEY"))
    response = chat.invoke(
        [
            AIMessage(content=AIPrompt),
            HumanMessage(content=UserPrompt)
        ])
    print(response)
    res = json.loads(response.content)
    res['score'] = float(res['score']) if res['score'] != None else float(0)
    return res

In [30]:
testimony = df.sample(1)['text'].values[0]
print(testimony)
response = gem_classifier(features2, testimony)
print(response)

differenceCollegeCfaConseil: On a beaucoup de technologie et d'atelier en général on voit les cours en techno et on les pratiques après 
content='{\n"avis": "oui",\n"score": "0.30",\n"justification": "Le témoignage évoque seulement un seul critère d\'intérêt positif, à savoir l\'apprentissage permet de se concentrer sur le métier qui correspond à l\'apprenti, avec des formations adaptées à ses besoins et sans cours superflus (Points positifs n°1). Les autres critères d\'intérêts ne sont pas abordés dans le témoignage."\n}' response_metadata={'token_usage': {'prompt_tokens': 1058, 'total_tokens': 1185, 'completion_tokens': 127}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-756479fe-a03f-4d57-a536-3d8f0d4509fc-0' usage_metadata={'input_tokens': 1058, 'output_tokens': 127, 'total_tokens': 1185}
{'avis': 'oui', 'score': 0.3, 'justification': "Le témoignage évoque seulement un seul critère d'intérêt positif, à savoir l'apprentissage permet de se concentrer sur le métier qui co

# Thématiques

In [ ]:
# Exposition des verbatims
def expose_function(testimony, categories=''):
    if categories == '':
        categories = {
            'Intégration et ambiance': 'oui|non',
            'Apprentissage du métier': 'oui|non',
            'Horaires': 'oui|non',
            'Rythme entreprise - établissement scolaire': 'oui|non',
            'Avoir moins de vacances': 'oui|non',
            'Journée-type en entreprise' : 'oui|non',
            'Ambiance établissement': 'oui|non',
            'Difficulté des cours': 'oui|non',
            'Enseignement proposé par établissement': 'oui|non',
            'Equipements mis à disposition': 'oui|non',
            'Accessibilité pour les personnes en situation de handicap': 'oui|non',
            'Charge de travail': 'oui|non',
            'Journée-type en établissement' : 'oui|non',
        }

    AIPrompt = f"""
        Indiquer si un témoignage aborde un ou plusieurs sujets en fonction des catégories suivantes:
        ```
        - {'|'.join(list(categories.keys()))}
        ```

        Votre réponse doit obligatoirement être au format JSON suivant:
        {str(categories)}

        Si le témoignage ne fournit pas assez d'informations ou avec un contenu non-significatif,
        renvoyer une réponse négative pour toutes les catégories.

        Votre réponse en JSON sans commentaire ou justification:
    """

    UserPrompt = f"""
        Voici un témoignage à catégoriser:
        ```
        {testimony}
        ```
    """
    chat = ChatMistralAI(mistral_api_key=os.environ['SIRIUS_MISTRAL_API_KEY']).bind(response_format={"type": "json_object"})
    response = chat.invoke(
        [
            AIMessage(content=AIPrompt),
            HumanMessage(content=UserPrompt)
        ])
    try:
        return json.loads(response.content)
    except:
        return {key:"non" for key in categories.keys()}

In [ ]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
samples['exposition'] = samples['text'].progress_apply(lambda x:expose_function(x))

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for i, row in samples.iterrows():
    print(f'"{row["text"]}"')
    for key, value in row['exposition'].items():
        print(f"\t- {key}: {value}")
    print()

"Tu es plus libre en tant qu’apprenti que lycéenne, cela te fait rentrer dans la vie active plus tôt."
	- Intégration et ambiance: non
	- Apprentissage du métier: oui
	- Horaires: non
	- Rythme entreprise - établissement scolaire: oui
	- Avoir moins de vacances: non
	- Journée-type en entreprise: non
	- Ambiance établissement: non
	- Difficulté des cours: non
	- Enseignement proposé par établissement: non
	- Equipements mis à disposition: non
	- Accessibilité pour les personnes en situation de handicap: non
	- Charge de travail: non
	- Journée-type en établissement: non

"Dit toi que tu est en apprentissage, tu est la pour apprendre un métier. Sois bien concentré sur ce que l'on t explique et sa ira si tu aimes se que tu fais."
	- Intégration et ambiance: non
	- Apprentissage du métier: oui
	- Horaires: non
	- Rythme entreprise - établissement scolaire: non
	- Avoir moins de vacances: non
	- Journée-type en entreprise: non
	- Ambiance établissement: non
	- Difficulté des cours: non
	- 

# Correction

In [ ]:
# Correction des verbatims
def correct_function(testimony='', instructions=''):
    FORMAT ="""
    {
        "correction": "texte corrigé",
        "modification": "oui|non si le texte corrigé est différent du texte d'origine"
        "justification": "justification de la correction"
    }
    """

    if instructions == '':
        instructions = """
        - Corrigez uniquement les fautes d'orthographes et de grammaires.
        - Ne pas reformuler le texte.
        - Ne pas remplacer des mots ou groupes de mots par des synonymes.
        - Conserver le même registre de langage.
        - Si le témoignage ne fournit pas assez d'informations ou est incompréhensible, ne rien corriger.
        """

    AIPrompt = f"""
        Vous êtes un correcteur automatique qui doit renvoyer une version corrigée de témoignages.

        Instructions:
        {instructions}

        Réponse attendue:
        Répondre uniquement selon le format suivant:
        {FORMAT}

        Votre réponse en objet JSON:
    """

    UserPrompt = f"""
        Voici un témoignage à corriger:
        ```
        {testimony}
        ```
    """

    chat = ChatMistralAI(
        model="mistral-large-latest",
        mistral_api_key=os.environ['SIRIUS_MISTRAL_API_KEY'],
        temperature=0
        ).bind(response_format={"type": "json_object"})

    response = chat.invoke(
        [
            AIMessage(content=AIPrompt),
            HumanMessage(content=UserPrompt)
        ])

    try:
        return json.loads(response.content)
    except:
        return {'correction': '', 'modification': 'non', 'justification': ''}

In [ ]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
samples['correction'] = samples['text'].progress_apply(lambda x:correct_function(x))

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for i, row in samples.iterrows():
    print(f'"{row["text"]}"')
    for key, value in row['correction'].items():
        print(f"\t- {key}: {value}")
    print()

"Tu es plus libre en tant qu’apprenti que lycéenne, cela te fait rentrer dans la vie active plus tôt."
	- correction: Tu es plus libre en tant qu’apprenti que lycéen, cela te fait rentrer dans la vie active plus tôt.
	- modification: oui
	- justification: Correction de 'lycéenne' en 'lycéen' pour accorder avec 'apprenti'.

"Dit toi que tu est en apprentissage, tu est la pour apprendre un métier. Sois bien concentré sur ce que l'on t explique et sa ira si tu aimes se que tu fais."
	- correction: Dis-toi que tu es en apprentissage, tu es là pour apprendre un métier. Sois bien concentré sur ce que l'on t'explique et ça ira si tu aimes ce que tu fais.
	- modification: oui
	- justification: Correction des fautes d'orthographe et de grammaire : 'Dit' -> 'Dis-toi', 'tu est' -> 'tu es', 'la' -> 'là', 't explique' -> 't'explique', 'sa' -> 'ça', 'se' -> 'ce'.

"L'apprentissage permet de prendre conscience de la réalité du métier vers lequel on s'oriente et de prendre son indépendance et des resp

# Anonymisation

In [ ]:
# Anonymisation des verbatims
def anonymize_function(testimony='', instructions=''):
    FORMAT ="""
    {
        "anonymisation": "texte modifié",
        "modification": "oui|non si le texte modifié est différent du texte d'origine"
        "justification": "justification de la modification"
    }
    """

    if instructions == '':
        instructions = """
        Tu es un modèle de langage spécialisé dans l'anonymisation de texte.
        Ton objectif est de remplacer uniquement les prénoms et noms par des pronoms personnels équivalents,
        Tu dois conserver les autres types de noms propres (métier, relation, rôle, qualité, etc.)

        Voici des exemples de transformation :
        - Jeanne et Paul vont à la montagne -> Ils vont à la montagne
        - Salut Jean, j'espère que tu vas bien ? -> Salut, j'espère que tu vas bien ?
        - Je n'aime pas ce que fait Bob, ce n'est pas très intéressant. -> Je n'aime pas ce qu'il fait, ce n'est pas très intéressant.
        - Ma mère adorait mon oncle Jean. -> Ma mère adorait mon oncle.
        - Le stade de Marseille ? C'est vraiment mieux que celui d'Antoine ! -> Marseille ? C'est vraiment mieux que le sien !
        - Je connais ma patronne depuis que je suis petit donc je lui ai demandé -> Je connais ma patronne depuis que je suis petit donc je lui ai demandé
        """

    AIPrompt = f"""
        {instructions}

        Réponse attendue:
        Répondre uniquement selon le format suivant:
        {FORMAT}

        Votre réponse en objet JSON:
    """

    UserPrompt = f"""
        Voici un témoignage à modifier:
        ```
        {testimony}
        ```
    """

    chat = ChatMistralAI(
        mistral_api_key=os.environ['SIRIUS_MISTRAL_API_KEY'],
        model = "mistral-large-latest",
        temperature=0
        ).bind(response_format={"type": "json_object"})

    response = chat.invoke(
        [
            AIMessage(content=AIPrompt),
            HumanMessage(content=UserPrompt)
        ])

    try:
        return json.loads(response.content)
    except:
        return {'anonymisation': '', 'modification': 'non', 'justification': ''}

In [ ]:
# Ajout de noms
import random
prenoms = ["Jean le plombier", 'Sabine', 'Madame Irma']
samples_with_names = samples['text'].apply(lambda x: f"Bonjour {random.choice(prenoms)}, j'espère que tu vas bien ? {x}")
samples_with_names = pd.DataFrame(samples_with_names)
samples_with_names

,text
8267,"Bonjour Sabine, j'espère que tu vas bien ? Tu ..."
11871,"Bonjour Jean le plombier, j'espère que tu vas ..."
12018,"Bonjour Jean le plombier, j'espère que tu vas ..."
12639,"Bonjour Madame Irma, j'espère que tu vas bien ..."
12103,"Bonjour Sabine, j'espère que tu vas bien ? Je ..."


In [ ]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
samples_with_names['anonymisation'] = samples_with_names['text'].progress_apply(lambda x:anonymize_function(x))

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for i, row in samples_with_names.iterrows():
    print(f'"{row["text"]}"')
    for key, value in row['anonymisation'].items():
        print(f"\t- {key}: {value}")
    print()

"Bonjour Sabine, j'espère que tu vas bien ? Tu es plus libre en tant qu’apprenti que lycéenne, cela te fait rentrer dans la vie active plus tôt."
	- anonymisation: Bonjour, j'espère que tu vas bien ? Tu es plus libre en tant qu’apprenti que lycéenne, cela te fait rentrer dans la vie active plus tôt.
	- modification: oui
	- justification: Le prénom 'Sabine' a été remplacé par un pronom personnel équivalent.

"Bonjour Jean le plombier, j'espère que tu vas bien ? Dit toi que tu est en apprentissage, tu est la pour apprendre un métier. Sois bien concentré sur ce que l'on t explique et sa ira si tu aimes se que tu fais."
	- anonymisation: Bonjour le plombier, j'espère que tu vas bien ? Dit toi que tu est en apprentissage, tu est la pour apprendre un métier. Sois bien concentré sur ce que l'on t explique et sa ira si tu aimes se que tu fais.
	- modification: oui
	- justification: Le prénom 'Jean' a été remplacé par un pronom personnel équivalent pour anonymiser le texte.

"Bonjour Jean le pl